In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import pandas as pd
import numpy as np
import geopandas as gpd
from pydantic import BaseModel
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt
import random

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

import networkx as nx
import ribasim

In [ ]:
%load_ext autoreload
%autoreload 2

Define base directory and results directory

In [ ]:
base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")

Define network name and load areas (discharge units: afwaterende eenheden)

In [ ]:
network_name = "zutphen_tki_netwerk"

areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_clip_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)

Create networkanalysis

In [ ]:
network = RibasimLumpingNetwork(name=network_name, areas_gdf=areas_gdf)

Select simulation sets and extract all data points

In [ ]:
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)

In [ ]:
network.get_network_data()

In [ ]:
# network.nodes_gdf[network.nodes_gdf.mesh1d_node_id.isin(['1334'])]

boundary

In [ ]:
import dfm_tools as dfmt
import hydrolib.core.dflowfm as hcdfm

In [ ]:
simulation_name = 'tki_zuthpen_berkel_basis.dsproj_data'

In [ ]:
#Load .bc-file using HydroLib object ForcingModel.

file_bc = f'{base_dir}\d-hydro\\{simulation_name}\FlowFM\input\FlowFM_boundaryconditions1d.bc'
# file_lat = r'C:\Users\NLTAND\OneDrive - Sweco AB\Algemeen-Tessa\Projecten\TKI oppervlaktewatermodule NHI\ribasim_lumping_data\d-hydro\tki_zuthpen_berkel_basis.dsproj_data\FlowFM\input\FlowFM_lateral_sources.bc'
forcingmodel_object = hcdfm.ForcingModel(file_bc)


In [ ]:
bc = pd.DataFrame([forcing.dict() for forcing in forcingmodel_object.forcing])

# convert dictionary with boundary type to columns
bc = pd.concat([bc.drop(['quantityunitpair'], axis=1), pd.DataFrame.from_records(bc['quantityunitpair'])[0].apply(pd.Series)], axis=1)

# merge boundary with nodes
boundaries_gdf = network.nodes_gdf.merge(bc,left_on = 'mesh1d_node_id', right_on = 'name')
boundaries_gdf = boundaries_gdf.drop(columns=['mesh1d_node_x','mesh1d_node_y','offset','factor','vertpositionindex','name', 'comments','datablock'])

In [ ]:
boundaries_gdf

In [ ]:
network.boundaries_gdf = boundaries_gdf
network.export_to_geopackage(output_dir=results_dir)